# Rarity Analyse 
**created by yuteng_zeng on 4/23/2023**

In [1]:
# install required library
#!pip install web3

In [11]:
# import libraries
import requests
import json
from web3 import Web3

import numpy as np
import os
import time
import h5py

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt

In [3]:
# get api_interfaces by tangyizhi
# API factors
API_key = 'PlIE5PSJaLBOTy67DORlQwY3MchgQxfP'
API_url = "https://eth-mainnet.g.alchemy.com/v2/PlIE5PSJaLBOTy67DORlQwY3MchgQxfP"
API_nft_url = 'https://eth-mainnet.g.alchemy.com/nft/v2/PlIE5PSJaLBOTy67DORlQwY3MchgQxfP'

# data collection functions
def getNFTsForCollection(contractAddress, withMetadata):
    url = API_url + '/getNFTsForCollection?contractAddress=' + contractAddress + '&withMetadata=' + str(withMetadata)
    r = requests.get(url, headers={"accept": "application/json"})
    return r
def getNFTSales(contractAddress, tokenid):
    # from block 0 to latest block, with ascending order
    url = API_url + '/getNFTSales?fromBlock=0&toBlock=latest&order=asc&contractAddress=' + contractAddress + '&tokenId=' + str(tokenid)
    r = requests.get(url, headers={"accept": "application/json"})
    return r

def getFloorPrice(contractAddress):
    url = API_nft_url + "/getFloorPrice?contractAddress=" + contractAddress
    r = requests.get(url, headers={"accept": "application/json"})
    return r

def computeRarity(contractAddress, tokenid):
    url = API_nft_url + '/computeRarity/?contractAddress=' + contractAddress + '&tokenId=' + str(tokenid)
    r = requests.get(url, headers={"accept": "application/json"})
    return r

def getMetaData(contractAddress, tokenid):
    url = API_nft_url + '/getNFTMetadata?contractAddress=' + contractAddress + '&tokenId=' + str(tokenid) + '&refreshCache=false'
    r = requests.get(url, headers={"accept": "application/json"})
    return r

def getNFTAttributes(contractAddress):
    url = API_nft_url + '/summarizeNFTAttributes?contractAddress=' + contractAddress
    r = requests.get(url, headers={"accept": "application/json"})
    return r

*The following is api test and further utils for data collection and prep*

In [4]:
response = json.loads(getNFTsForCollection('0x60E4d786628Fea6478F785A6d7e704777c86a7c6', True).content)
print(response['nfts'][0])

{'contract': {'address': '0x60e4d786628fea6478f785a6d7e704777c86a7c6'}, 'id': {'tokenId': '0x0000000000000000000000000000000000000000000000000000000000000000', 'tokenMetadata': {'tokenType': 'ERC721'}}, 'title': '', 'description': '', 'tokenUri': {'gateway': 'https://boredapeyachtclub.com/api/mutants/0', 'raw': 'https://boredapeyachtclub.com/api/mutants/0'}, 'media': [{'gateway': 'https://nft-cdn.alchemy.com/eth-mainnet/c480d814f8a4e2b5f02dd3aa189ca742', 'thumbnail': 'https://res.cloudinary.com/alchemyapi/image/upload/thumbnailv2/eth-mainnet/c480d814f8a4e2b5f02dd3aa189ca742', 'raw': 'ipfs://QmURua8bNrAwX76Tp6G9t6Lospdxyt61JGy3UsXY7skfR1', 'format': 'png', 'bytes': 577530}], 'metadata': {'image': 'ipfs://QmURua8bNrAwX76Tp6G9t6Lospdxyt61JGy3UsXY7skfR1', 'attributes': [{'value': 'M1 Purple', 'trait_type': 'Background'}, {'value': 'M1 Cheetah', 'trait_type': 'Fur'}, {'value': 'M1 Scumbag', 'trait_type': 'Eyes'}, {'value': 'M1 Toga', 'trait_type': 'Clothes'}, {'value': 'M1 Bored Unshaven', 

In [5]:
# check the attribute lists.
response = json.loads(getNFTAttributes('0x60E4d786628Fea6478F785A6d7e704777c86a7c6').content)
print(response['summary'].keys())

dict_keys(['Fur', 'Eyes', 'Background', 'Mouth', 'Clothes', 'Earring', 'Hat', 'Name'])


In [6]:
response = json.loads(computeRarity('0x60E4d786628Fea6478F785A6d7e704777c86a7c6',0).content)
print(response)

[{'value': 'M1 Purple', 'trait_type': 'Background', 'prevalence': 0.09621621621621622}, {'value': 'M1 Cheetah', 'trait_type': 'Fur', 'prevalence': 0.03099099099099099}, {'value': 'M1 Scumbag', 'trait_type': 'Eyes', 'prevalence': 0.016885456885456885}, {'value': 'M1 Toga', 'trait_type': 'Clothes', 'prevalence': 0.01662805662805663}, {'value': 'M1 Bored Unshaven', 'trait_type': 'Mouth', 'prevalence': 0.11613899613899614}]


In [7]:
response = json.loads(getNFTSales('0x60E4d786628Fea6478F785A6d7e704777c86a7c6',555).content)
print(response['nftSales'][-1]['sellerFee'])
print(response['nftSales'][-1]['protocolFee'])
print(response['nftSales'][-1]['royaltyFee'])

{'amount': '23740500000000000000', 'tokenAddress': '0x0000000000000000000000000000000000000000', 'symbol': 'ETH', 'decimals': 18}
{'amount': '624750000000000000', 'tokenAddress': '0x0000000000000000000000000000000000000000', 'symbol': 'ETH', 'decimals': 18}
{'amount': '624750000000000000', 'tokenAddress': '0x0000000000000000000000000000000000000000', 'symbol': 'ETH', 'decimals': 18}


In [8]:
# get the sell price from the api
def get_price(contractAddress, tokenid, roundto):
    response = json.loads(getNFTSales(contractAddress,tokenid).content)
    res = int(response['nftSales'][-1]['sellerFee']['amount'])/ (10**int(response['nftSales'][-1]['sellerFee']['decimals']))
    res = round(res, roundto)
    return res

print(get_price('0x60E4d786628Fea6478F785A6d7e704777c86a7c6',555,5))
print(get_price('0x60E4d786628Fea6478F785A6d7e704777c86a7c6',28905,5))

23.7405
106.875


In [9]:
# get feature and labels for building the dataset
def get_monkey(contractAddress, tokenid, roundto):
    # initialize all attributes to one, which means least rare
    feature = {'Fur':1, 'Eyes':1, 'Background':1, 'Mouth':1, 'Clothes':1, 'Earring':1, 'Hat':1, 'Name':1}
    feature_response = json.loads(computeRarity(contractAddress,tokenid).content)
    #print(feature_response)
    for i in feature_response:
        feature[i['trait_type']] = round(1 / i['prevalence'], roundto)
        
    # get price
    price = None
    try:
        price = get_price(contractAddress, tokenid, roundto)
    except:
        pass
    return feature, price

there are monkeys that has never been traded, thus *try* get the price, 

if price donnot exist then the price is set to **None**

In [15]:
print(get_monkey('0x60E4d786628Fea6478F785A6d7e704777c86a7c6',2,5))
print(get_monkey('0x60E4d786628Fea6478F785A6d7e704777c86a7c6',10000,5)[1] is None)

({'Fur': 9.95643, 'Eyes': 7.71752, 'Background': 10.76177, 'Mouth': 5.86327, 'Clothes': 33.31904, 'Earring': 1, 'Hat': 99.61538, 'Name': 1}, 20.9)
True


make the dataset using the template ['Fur', 'Eyes', 'Background', 'Mouth', 'Clothes', 'Earring', 'Hat', 'Name'] as **features**

seller price as **labels** to predict


In [21]:
# now gathering the data and build dataset

attribute_list = ['Fur', 'Eyes', 'Background', 'Mouth', 'Clothes', 'Earring', 'Hat', 'Name']
features = []
labels = []
token_id = 1
while len(labels) <= 150:    
    feature, price = get_monkey('0x60E4d786628Fea6478F785A6d7e704777c86a7c6',token_id, 5)
    if price is not None:
        features.append([feature[x] for x in attribute_list])
        labels.append(price)
    token_id += 1
    time.sleep(0.01)
print(features, labels)

[[20.90958, 40.21739, 11.1, 25.9, 56.63265, 1, 84.45652, 1], [9.95643, 7.71752, 10.76177, 5.86327, 33.31904, 1, 99.61538, 1], [17.45283, 41.24204, 10.1489, 8.61037, 1, 1, 52.35849, 1], [9.80565, 40.21739, 10.8459, 31.79214, 51.11842, 1, 85.57269, 1], [10.85802, 17.43716, 10.3766, 98.60406, 73.30189, 29.25452, 35.51188, 1], [33.95979, 33.03571, 10.3766, 18.58852, 92.5, 1, 23.0975, 1], [22.27638, 15.18765, 10.1489, 5.86327, 66.52397, 1, 53.66022, 1], [33.95979, 7.71752, 10.3766, 98.60406, 38.77246, 1, 45.06961, 1], [26.35685, 51.8, 10.1489, 8.61037, 63.06818, 1, 1, 1], [68.88298, 109.74576, 32.10744, 61.86306, 183.25472, 46.69471, 1, 1], [9.80565, 18.80445, 10.8459, 50.71802, 59.22256, 1, 84.45652, 1], [28.07081, 24.77679, 11.1, 18.58852, 60.13932, 1, 67.44792, 1], [34.01926, 181.54206, 29.88462, 97.125, 236.89024, 1, 127.79605, 1], [57.8125, 7.71752, 10.8459, 5.86327, 60.13932, 1, 67.44792, 1], [33.95979, 27.71041, 11.1, 50.85079, 56.63265, 1, 57.47041, 1], [34.01926, 123.72611, 32.7020

In [35]:
# partially acquire data... getting too fast would cause api denial
while len(labels) < 1200:    
    feature, price = get_monkey('0x60E4d786628Fea6478F785A6d7e704777c86a7c6',token_id, 5)
    if price is not None:
        features.append([feature[x] for x in attribute_list])
        labels.append(price)
    token_id += 1
    time.sleep(0.01)
print(features, labels)

[[20.90958, 40.21739, 11.1, 25.9, 56.63265, 1, 84.45652, 1], [9.95643, 7.71752, 10.76177, 5.86327, 33.31904, 1, 99.61538, 1], [17.45283, 41.24204, 10.1489, 8.61037, 1, 1, 52.35849, 1], [9.80565, 40.21739, 10.8459, 31.79214, 51.11842, 1, 85.57269, 1], [10.85802, 17.43716, 10.3766, 98.60406, 73.30189, 29.25452, 35.51188, 1], [33.95979, 33.03571, 10.3766, 18.58852, 92.5, 1, 23.0975, 1], [22.27638, 15.18765, 10.1489, 5.86327, 66.52397, 1, 53.66022, 1], [33.95979, 7.71752, 10.3766, 98.60406, 38.77246, 1, 45.06961, 1], [26.35685, 51.8, 10.1489, 8.61037, 63.06818, 1, 1, 1], [68.88298, 109.74576, 32.10744, 61.86306, 183.25472, 46.69471, 1, 1], [9.80565, 18.80445, 10.8459, 50.71802, 59.22256, 1, 84.45652, 1], [28.07081, 24.77679, 11.1, 18.58852, 60.13932, 1, 67.44792, 1], [34.01926, 181.54206, 29.88462, 97.125, 236.89024, 1, 127.79605, 1], [57.8125, 7.71752, 10.8459, 5.86327, 60.13932, 1, 67.44792, 1], [33.95979, 27.71041, 11.1, 50.85079, 56.63265, 1, 57.47041, 1], [34.01926, 123.72611, 32.7020

In [1]:
print(token_id)

NameError: name 'token_id' is not defined

In [34]:
print(len(features), len(labels))

1000 1000
